In [43]:

import os
import sys
import random
import math
import time

import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import myVisualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [3]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Create Model and Load Trained Weights

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

my_class_ids = [class_names.index('person'), class_names.index('car'),
                class_names.index('bottle'), class_names.index('chair'),
                class_names.index('toilet'), class_names.index('knife')]

## Run Object Detection cycling on the images

In [6]:
img_path = "C:\\Users\\danie\\OneDrive - Scuola Superiore Sant'Anna\\PyCharm\\photos_simple"
photos = os.listdir(img_path)
r_list = [None] * len(photos)
savepath = "C:\\Users\\danie\\OneDrive - Scuola Superiore Sant'Anna\\PyCharm\\masks_simple_2"

To generate a mask only if the detected object belongs to the category of interest

In [44]:
color = [1, 1, 1]
alpha = 1
elapsed_time_SingleMask = []

for img in range(len(photos)):
    image = skimage.io.imread(img_path + "\\" + photos[img])

    # calculate elapsed time
    t0 = time.clock()

    # Run detection and update the results list
    results = model.detect([image], verbose=1)
    r_list[img] = results

    r = results[0]
    masks = r['masks']
    boxes = r['rois']
    class_ids = r['class_ids']

    # Number of instances
    N = boxes.shape[0]
    # Dimensions of the picture
    height, width = image.shape[:2]
    # Black background
    mask_img = np.zeros((height, width, 3), dtype = "uint8")

    for i in range(N):

        for my_id in my_class_ids:
            if class_ids[i] == my_id:
                mask = masks[:, :, i]

                for c in range(3):
                    mask_img[:, :, c] = np.where(mask == 1,
                                            mask_img[:, :, c] *
                                            (1 - alpha) + alpha * color[c] * 255,
                                            mask_img[:, :, c])
    t1 = time.clock() - t0
    elapsed_time_SingleMask.append(t1)
 #           cv2.imwrite(savepath + "\\" + photos[img], mask_img)
          #  print('Done 1')

Processing 1 images
image                    shape: (768, 576, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (417, 626, 3)         min:    0.00000  max:  252.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  140.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (1689, 1064, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

To generate a mask for every detected object

In [45]:
savepath = "C:\\Users\\danie\\OneDrive - Scuola Superiore Sant'Anna\\PyCharm\\masks_simple_more"
color = [1, 1, 1]
alpha = 1
elapsed_time_MoreMasks = []

for img in range(len(photos)):
    image = skimage.io.imread(img_path + "\\" + photos[img])

    # calculate elapsed time
    t0 = time.clock()

    # Run detection and update the results list
    results = model.detect([image], verbose=1)
    r_list[img] = results

    r = results[0]
    masks = r['masks']
    boxes = r['rois']
    class_ids = r['class_ids']

    # Number of instances
    N = boxes.shape[0]
    # Dimensions of the picture
    height, width = image.shape[:2]
    # Create N black backgrounds
    bg_list = [];
    for obj in range(N):
       # globals()['mask_img_%s' % str(obj)] = np.zeros((height, width, 3), dtype = "uint8")
        bg_list.append(np.zeros((height, width, 3), dtype = "uint8"))

    for i in range(N):
        mask = masks[:, :, i]

        for c in range(3):
            bg_list[i][:, :, c] = np.where(mask == 1,
                                    bg_list[i][:, :, c] *
                                    (1 - alpha) + alpha * color[c] * 255,
                                    bg_list[i][:, :, c])
    t1 = time.clock() - t0
    elapsed_time_MoreMasks.append(t1)
 #       cv2.imwrite(savepath + "\\mask" + str(i) + photos[img], bg_list[i])

Processing 1 images
image                    shape: (768, 576, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (417, 626, 3)         min:    0.00000  max:  252.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  140.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (1689, 1064, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1